In [3]:
import numpy as np


In [75]:
# Data generation
n = 100
true_mu1 = np.array([1, 1]) #mean for Gaussian Class A
true_mu2 = np.array([-1, -1]) #mean for Gaussian Class B
sigma = np.eye(2) #2x2

prob_A = (n/(2*n)) #prior prob for each Class in this case I put it to 50/50
prob_B = 1 - prob_A

X1 = np.random.multivariate_normal(true_mu1, sigma, n) #Class A
X2 = np.random.multivariate_normal(true_mu2, sigma, n) #Class B

X = np.vstack([X1, X2]) # 2D array of all data points
Y = np.hstack([np.ones(n), np.zeros(n)]) # Vector denoting which class each row in X belongs to

In [72]:
# Gaussian Generative Model
est_mu1 = X[Y==1].mean(axis=0) # Compute mean of all points in Class A --> return mean vector of [x1, x2]
est_mu2 = X[Y==0].mean(axis=0) # Compute mean of all points in Class B --> return mean vector of [x1, x2]

# This computes posterior probability for some Class
def posterior_prob(x_data, est_mu1, est_mu2, sigma, prob_A, prob_B):
    # Quadratic terms in x from the exponents of the Gaussian densities cancel since each Class shares covariance matrix 
    # Resulting in a linear function of x in the argument of the logistic sigmoid
    # sigmoid(a) = sigmoid(w^Tx + w0)
    w = np.linalg.inv(sigma) @ (est_mu1 - est_mu2) #2x1
    w0 = (-1/2)*np.transpose(est_mu1) @ np.linalg.inv(sigma) @ (est_mu1) + \
    (1/2)*np.transpose(est_mu2) @ np.linalg.inv(sigma) @ (est_mu2) + \
    np.log(prob_A/prob_B)

    a = x_data @ w + w0
    
    return 1/(1+np.exp(-a))

In [76]:
# Gaussian Generative Model Classification Performance
y_pred = (posterior_prob(X, est_mu1, est_mu2, sigma, prob_A, prob_B) > 0.5).astype(int)
accuracy = 1 - np.abs(np.sum(y_pred) - np.sum(Y))/np.sum(Y)

print(f"Accuracy: {accuracy:.2f}")

Accuracy: 0.93


In [ ]:
# IRLS Algorithm